<a href="https://colab.research.google.com/github/aetev/Hearth-Stone-Python-Simulator/blob/main/card_enc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [142]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [143]:
import json
import tensorflow as tf
import re
from tensorflow.keras.layers import Embedding, TextVectorization
# Assuming AtomicCards.json is in your Google Drive, adjust the path if necessary
file_path = '/content/drive/MyDrive/MTGdata/AtomicCards.json'


with open(file_path, 'r') as f:
  data = json.load(f)['data']

In [144]:
def filter_legal_cards(data):
    legal_commander_data = {}
    for card_name, card_data in data.items():
        try:
            if card_data[0]['legalities']['commander'] == 'Legal':
                legal_commander_data[card_name] = card_data[0] # Store only the first element of card_data
        except (KeyError, IndexError):
            pass  # Or print a message, log it, etc.
    return legal_commander_data

filtered_data = filter_legal_cards(data)


In [203]:
items = list(filtered_data.items())

text_list = []
for row in items[:100]:
  second_key, second_value = row
  if second_value['text'] != None:
    text_list.append(second_value['text'])


In [204]:
class TextCleaningLayer(tf.keras.layers.Layer):
    def __init__(self,sentence_list, **kwargs):
        super(TextCleaningLayer, self).__init__(**kwargs)
        self.vectorizer = TextVectorization(standardize=None, output_mode='int',output_sequence_length=150)
        self.vectorizer.adapt(self.clean_text(sentence_list))
        self.embedding_layer = Embedding(input_dim=3000, output_dim=100)

    def clean_text(self, text):

        #text = card['text']
        #card_name = card['name']


        # Convert input to string tensor if it's not already
        if not isinstance(text, tf.Tensor):
            text = tf.convert_to_tensor(text, dtype=tf.string)

        cleaned_text = tf.strings.lower(text)
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\n', ' ')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '[\/\.]', ' ')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '[\"—"]', ' ')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\{|\}', ' ')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\([^)]*\)', '')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '[^\w\s+/\-]', '')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\s+', ' ')
        return cleaned_text

    def call(self, inputs):
        cleaned_text = self.clean_text(inputs)
        vectorized_text = self.vectorizer(cleaned_text)
        embedded_text = self.embedding_layer(vectorized_text)
        return cleaned_text


# Example usage with a batch of texts
batch_texts = tf.constant(text_list)
cleaned_text_layer = TextCleaningLayer(batch_texts)
cleaned_batch = cleaned_text_layer.call(batch_texts)
for i in range(len(cleaned_batch)):
  tf.print(cleaned_batch[i])




this creature enters tapped when this creature dies open an attraction whenever you roll to visit your attractions if you roll a 6 you may return this card from your graveyard to the battlefield 
when this creature enters from anywhere other than a graveyard or exile if its on the battlefield and you control 9 or fewer creatures named name sticker goblin roll a 20-sided die 1-6 add r r r r 7-14 add r r r r r 15-20 add r r r r r r 
equipped creature gets +2 +2 equip 3 
tap up to two target creatures whenever you put a name sticker on a creature you may return this card from your graveyard to your hand 
when this enchantment enters create a 1 1 white human creature token a 1 1 blue merfolk creature token and a 1 1 red goblin creature token then secretly choose human merfolk or goblin sacrifice this enchantment reveal the creature type you chose if target attacking creature token is the chosen type put three +1 +1 counters on it and it gains deathtouch until end of turn 
casualty 1 look a

In [205]:
print(cleaned_text_layer.vectorizer.get_vocabulary())

['', '[UNK]', 'creature', 'this', 'a', 'you', 'of', 'the', 'target', 'card', 'to', 'your', '1', 'or', 'when', 'that', 'and', 'turn', 'enters', 'control', 'it', 'from', '+1', 'may', 'flying', 't', 'r', 'until', 'player', 'an', 'on', 'if', 'put', 'graveyard', 'end', 'each', 'choose', 'cast', 'with', 'whenever', 'its', 'creatures', 'one', 'life', 'exile', 'draw', 'cards', 'as', 'add', 'spell', 'dies', 'damage', 'token', 'return', 'mana', 'hand', 'at', 'w', 'two', 'library', 'has', 'cant', 'battlefield', '3', '2', 'land', 'in', 'gain', 'enchantment', 'enchanted', 'create', 'counter', 'artifact', 'all', '+2', 'x', 'u', 'permanent', 'deals', 'spells', 'other', 'opponent', 'nonland', 'next', 'have', 'for', 'discard', 'destroy', 'beginning', 'be', 'b', 'any', 'would', 'toughness', 'top', 'then', 'tap', 'sacrifice', 'power', 'opponents', 'number', 'gets', 'cost', 'additional', 'ability', 'white', 'upkeep', 'up', 'tapped', 'sorcery', 'reveal', 'owners', 'instant', 'equal', 'do', 'discards', 'cou